In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_data = pd.read_csv("/kaggle/input/titanic/train.csv")
train_data.head()

In [ ]:
test_data = pd.read_csv("/kaggle/input/titanic/test.csv")
test_data.head()

In [ ]:
train_data['Age'].dtype

In [ ]:
women = train_data.loc[train_data.Embarked == 'S']["Survived"]
rate_women = sum(women)/len(women)

print("% of women who survived:", rate_women)

In [ ]:
men = train_data.loc[train_data.Embarked == 'C']["Survived"]
rate_men = sum(men)/len(men)

print("% of men who survived:", rate_men)

In [ ]:
train_data["Relatives"] = train_data["SibSp"]+train_data["Parch"]
train_data.head()

In [ ]:
test_data["Relatives"] = test_data["SibSp"] + test_data["Parch"]
test_data.head()

In [ ]:
colunmAgeTrain = train_data["Age"]
mediaAgeTrain = np.mean(colunmAgeTrain)
print(mediaAgeTrain)

In [ ]:
colunmAgeTest = test_data["Age"]
mediaAgeTest = np.mean(colunmAgeTest)
print(mediaAgeTest)

In [ ]:
train_data['Age'] = train_data['Age'].replace({np.nan:mediaAgeTrain})

In [ ]:
test_data['Age'] = train_data['Age'].replace({np.nan:mediaAgeTest})

In [ ]:
train_data["Bebe"] = train_data['Age'] <= 5.0
train_data["Ninio"] = (train_data['Age'] <= 6.0) & (train_data['Age'] <= 14.0)
train_data["Joven"] = (train_data['Age'] >= 15.0) & (train_data['Age'] <= 30.0)
train_data["Adulto"] = (train_data['Age'] >= 31.0) & (train_data['Age'] <= 69.0)
train_data["Anciano"] = train_data['Age'] >= 70.0

train_data.head()

In [ ]:
train_data['Bebe'] = train_data['Bebe'].replace({True:1, False:0})
train_data['Ninio'] = train_data['Ninio'].replace({True:1, False:0})
train_data['Joven'] = train_data['Joven'].replace({True:1, False:0})
train_data['Adulto'] = train_data['Adulto'].replace({True:1, False:0})
train_data['Anciano'] = train_data['Anciano'].replace({True:1, False:0})

train_data.head()

In [ ]:
test_data["Bebe"] = test_data['Age'] <= 5.0
test_data["Ninio"] = (test_data['Age'] <= 6.0) & (test_data['Age'] <= 14.0)
test_data["Joven"] = (test_data['Age'] >= 15.0) & (test_data['Age'] <= 30.0)
test_data["Adulto"] = (test_data['Age'] >= 31.0) & (test_data['Age'] <= 69.0)
test_data["Anciano"] = test_data['Age'] >= 70.0

test_data.head()

In [ ]:
test_data['Bebe'] = test_data['Bebe'].replace({True:1, False:0})
test_data['Ninio'] = test_data['Ninio'].replace({True:1, False:0})
test_data['Joven'] = test_data['Joven'].replace({True:1, False:0})
test_data['Adulto'] = test_data['Adulto'].replace({True:1, False:0})
test_data['Anciano'] = test_data['Anciano'].replace({True:1, False:0})

test_data.head()

In [ ]:
tieneNaN = train_data["Age"].isna().values.any()
sumaNaNAge = train_data["Age"].isna().sum()
sumaNaN = train_data["Age"].sum()

print("¿Tiene valor NaN?",tieneNaN," \nNº NaN: ",sumaNaNAge)
print(sumaNaN)

In [ ]:
tieneNaNTest = train_data["Age"].isna().values.any()
sumaNaNAgeTest = train_data["Age"].isna().sum()
sumaNaNTest = train_data["Age"].sum()

print("¿Tiene valor NaN?",tieneNaNTest," \nNº NaN: ",sumaNaNAgeTest)
print(sumaNaNTest)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

y = train_data["Survived"]

features = ["Pclass", "Sex", "Embarked", "Relatives", "Bebe", "Ninio", "Joven", "Adulto", "Anciano"]
X = pd.get_dummies(train_data[features])
X_test = pd.get_dummies(test_data[features])

model = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=1)
model.fit(X, y)
predictions = model.predict(X_test)

output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': predictions})
output.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")